In [1]:
import os
import re

import numpy as np
import pandas as pd

# os.chdir("/home/yujia/Project/2023-07-20-individual_MR/dat/02_base_data/")

In [2]:
LDL = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/download/without_UKB_LDL_INV_SAS_1KGP3_ALL.meta.singlevar.results.gz", sep="\t", compression = 'gzip')

print("Total SNP in the GWAS data: ")
print(LDL.shape)

Total SNP in the GWAS data: 
(15268864, 14)


In [3]:
LDL

,rsID,CHROM,POS_b37,REF,ALT,N,N_studies,POOLED_ALT_AF,EFFECT_SIZE,SE,pvalue_neg_log10,pvalue,pvalue_neg_log10_GC,pvalue_GC
0,rs189800799,1,701835,T,C,2226,2,0.03370,0.045730,0.094141,0.202639,0.627,0.199134,0.632
1,rs373285923,1,712547,G,C,2226,2,0.03410,0.077800,0.115543,0.300399,0.501,0.294944,0.507
2,rs12071806,1,712762,T,G,2226,2,0.03890,0.120097,0.088868,0.753093,0.177,0.737490,0.183
3,NaN,1,713044,C,<CN2>,2226,2,0.03350,0.054683,0.094697,0.249005,0.564,0.244590,0.569
4,rs200310424,1,713131,A,AT,2226,2,0.03370,0.048563,0.090536,0.227908,0.592,0.223911,0.597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15268859,rs9616834,22,51222965,G,A,2604,3,0.20000,0.003440,0.057589,0.021198,0.952,0.020878,0.953
15268860,rs373624689,22,51228439,G,A,8834,5,0.04480,-0.034475,0.041268,0.394166,0.403,0.386737,0.410
15268861,rs144549712,22,51229855,G,A,8834,5,0.13200,0.017862,0.029039,0.268826,0.538,0.264014,0.544
15268862,rs200189535,22,51235959,T,C,8834,5,0.27200,0.000403,0.025866,0.005429,0.988,0.005348,0.988


In [4]:
# rename the data (REF: non-effect allele; ALT: effect allele)
LDL.rename(columns={"CHROM":"CHR", "POS_b37":"BP", "rsID":"SNP", "REF":"A2", "ALT":"A1", "POOLED_ALT_AF":"EAF", "EFFECT_SIZE":"BETA", "SE":"SE", "pvalue":"P", "N":"N"}, inplace=True)

# convert EAF to MAF
LDL['MAF'] = LDL['EAF'].apply(lambda x : x if x <= 0.5 else 1-x)

# Uppercase the A1 and A2
LDL['A1'] = LDL['A1'].str.upper()
LDL['A2'] = LDL['A2'].str.upper()

In [5]:
LDL

,SNP,CHR,BP,A2,A1,N,N_studies,EAF,BETA,SE,pvalue_neg_log10,P,pvalue_neg_log10_GC,pvalue_GC,MAF
0,rs189800799,1,701835,T,C,2226,2,0.03370,0.045730,0.094141,0.202639,0.627,0.199134,0.632,0.03370
1,rs373285923,1,712547,G,C,2226,2,0.03410,0.077800,0.115543,0.300399,0.501,0.294944,0.507,0.03410
2,rs12071806,1,712762,T,G,2226,2,0.03890,0.120097,0.088868,0.753093,0.177,0.737490,0.183,0.03890
3,NaN,1,713044,C,<CN2>,2226,2,0.03350,0.054683,0.094697,0.249005,0.564,0.244590,0.569,0.03350
4,rs200310424,1,713131,A,AT,2226,2,0.03370,0.048563,0.090536,0.227908,0.592,0.223911,0.597,0.03370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15268859,rs9616834,22,51222965,G,A,2604,3,0.20000,0.003440,0.057589,0.021198,0.952,0.020878,0.953,0.20000
15268860,rs373624689,22,51228439,G,A,8834,5,0.04480,-0.034475,0.041268,0.394166,0.403,0.386737,0.410,0.04480
15268861,rs144549712,22,51229855,G,A,8834,5,0.13200,0.017862,0.029039,0.268826,0.538,0.264014,0.544,0.13200
15268862,rs200189535,22,51235959,T,C,8834,5,0.27200,0.000403,0.025866,0.005429,0.988,0.005348,0.988,0.27200


In [6]:
# reorder the column
cols = ["CHR", "BP", "SNP", "A1", "A2", "N", "SE", "P", "BETA", "MAF", "EAF"]
LDL = LDL[cols]

# Remove NaN row
LDL = LDL.dropna()

# convert to correct column type
LDL = LDL.astype({'CHR':'int', 'BP':'int', "SNP":'str', 'A1':'str', 'A2':'str', 'N':'int', 'SE':'float', 'P':'float', 'BETA':'float', 'MAF':'float', 'EAF':'float'})

# remove SNP without SNPID
LDL = LDL[LDL['SNP'].str.startswith('rs')]

# save the formatted results
LDL.to_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL_formated_nadrop.tsv.gz", sep="\t", index=False)

print("SNP left after basic QC: ")
print(LDL.shape)

SNP left after basic QC: 
(15182712, 11)


In [7]:
LDL

,CHR,BP,SNP,A1,A2,N,SE,P,BETA,MAF,EAF
0,1,701835,rs189800799,C,T,2226,0.094141,0.627,0.045730,0.03370,0.03370
1,1,712547,rs373285923,C,G,2226,0.115543,0.501,0.077800,0.03410,0.03410
2,1,712762,rs12071806,G,T,2226,0.088868,0.177,0.120097,0.03890,0.03890
4,1,713131,rs200310424,AT,A,2226,0.090536,0.592,0.048563,0.03370,0.03370
5,1,714019,rs114983708,G,A,2226,0.085192,0.183,0.113429,0.03890,0.03890
...,...,...,...,...,...,...,...,...,...,...,...
15268859,22,51222965,rs9616834,A,G,2604,0.057589,0.952,0.003440,0.20000,0.20000
15268860,22,51228439,rs373624689,A,G,8834,0.041268,0.403,-0.034475,0.04480,0.04480
15268861,22,51229855,rs144549712,A,G,8834,0.029039,0.538,0.017862,0.13200,0.13200
15268862,22,51235959,rs200189535,C,T,8834,0.025866,0.988,0.000403,0.27200,0.27200


In [8]:
# =============================
# Step 2: remove SNPs with MAF < 0.01
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL_formated_nadrop.tsv.gz |" + \
      "awk 'NR==1 || ($10 > 0.01) {print}' |" + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.standardGWASQC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.standardGWASQC.gz | wc -l"
print("SNP left after removing SNPs with MAF < 0.01: ")
print(os.popen(cmd).read())

SNP left after removing SNPs with MAF < 0.01: 
9274512



In [9]:
# =============================
# Step 3: remove duplicate SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.standardGWASQC.gz |" + \
      "awk '{seen[$3]++; if(seen[$3]==1){ print}}' |" + \
      "gzip - > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.nodup.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.nodup.gz | wc -l"
print("SNP left after removing duplicate SNPs: ")
print(os.popen(cmd).read())

SNP left after removing duplicate SNPs: 
9254313



In [10]:
# =============================
# Step 4: keep non-ambiguous SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.nodup.gz | " + \
      "awk '!( ($4==\"A\" && $5==\"T\") || ($4==\"T\" && $5==\"A\") || ($4==\"G\" && $5==\"C\") || ($4==\"C\" && $5==\"G\")) {print}' | " + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.QC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_SAS/LDL.QC.gz | wc -l"
print("SNP left after removing ambiguous SNPs: ")
print(os.popen(cmd).read())

SNP left after removing ambiguous SNPs: 
7964877

